In [15]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import os

In [16]:
gt_data = pd.read_json(f"./images/configurations.json")
len(gt_data)

120

In [17]:
# text_image_0a5a0d82-477d-47e2-a504-861c5952fe98-gemini-output.md
gt_data["gemini-output-file"] = gt_data["image_name"].apply(
    lambda x: "./images/" + x.replace(".png", "") + "-gemini-output.md"
)

# Read the content of each file for the gemini output and make a new column, ignoring missing files
gt_data["gemini-output-raw"] = gt_data["gemini-output-file"].apply(
    lambda x: (open(x, "r").read() if os.path.exists(os.path.join(x)) else None)
)

In [18]:
# drop rows with missing gemini output
gt_data = gt_data.dropna(subset=["gemini-output-raw"])

In [19]:
len(gt_data)

120

In [20]:
def extract_marked_text(text):
    # First try to extract text between **
    parts = text.split("**")
    if len(parts) > 1:
        # Extract the first marked text which is likely the number or description of intersections
        marked_text = parts[1].lower()
        # Map textual numbers to integer values
        number_mapping = {
            "zero": 0,
            "one": 1,
            "two": 2,
            "three": 3,
            "four": 4,
            "five": 5,
            "six": 6,
            "seven": 7,
            "eight": 8,
            "nine": 9,
            "ten": 10,
        }
        # Check if the marked text is a digit and return the integer if true
        if marked_text.isdigit():
            return int(marked_text)
        # Return the corresponding integer if the text is a known number, otherwise return the text
        return number_mapping.get(marked_text, marked_text)
    else:
        # If not found, return a default value indicating no marked text was found
        return "marker_not_found"


gt_data["predicted"] = gt_data["gemini-output-raw"].apply(extract_marked_text)

In [21]:
gt_data["predicted"].value_counts()

predicted
5    54
2    30
3    30
4     6
Name: count, dtype: int64

In [22]:
cleaned_data = gt_data.copy()
# drop squares column
cleaned_data = cleaned_data.drop(columns=["squares"])

In [23]:
# Ensure both columns are of integer type before comparison
cleaned_data["depth"] = cleaned_data["depth"].astype(int)
cleaned_data["predicted"] = cleaned_data["predicted"].astype(int)

cleaned_data["is_prediction_correct"] = (
    cleaned_data["depth"] == cleaned_data["predicted"]
)
# Calculate accuracy
accuracy = cleaned_data["is_prediction_correct"].mean()
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 80.00%


In [24]:
len(cleaned_data)

120

In [25]:
# Assuming 'cleaned_data' DataFrame has columns 'line_thickness', 'num_intersections', and 'extracted_number'

# Convert line_thickness to an appropriate numeric type if necessary
cleaned_data["line_thickness"] = cleaned_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = cleaned_data.groupby("line_thickness").apply(
    lambda df: (df["is_prediction_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_71097/1445926420.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = cleaned_data.groupby("line_thickness").apply(


line_thickness
2    80.0
3    80.0
4    80.0
dtype: float64

In [26]:
# Convert line_thickness to an appropriate numeric type if necessary
cleaned_data["depth"] = cleaned_data["depth"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = cleaned_data.groupby("depth").apply(
    lambda df: (df["is_prediction_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_71097/3496425564.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = cleaned_data.groupby("depth").apply(


depth
2    100.0
3    100.0
4     20.0
5    100.0
dtype: float64